<h1> ______________________________________ </h1>

<h1> Fiber barrel simulation analysis </h1>
<h1> ______________________________________ </h1>

<p style="font-size: 17px; color: black;"> In this Notebook we analyse the results from photons generated in 10000 MC events simulated in a Fiber barrel with photosensors in one end and Aluminium in the other end. </p>


<p style="font-size: 17px; color: black;"> For this, we first run some test to check that the MC output is indeed the type of events we wanted to simulate, later we calculate the system efficiency. </p>



<h1> ____________ </h1>
<h2> This version </h2>
<h1> ____________ </h1>

-  <p style="font-size: 17px; color: black;"> 10K events. </p>
-  <p style="font-size: 17px; color: black;"> Using perfect photosensors. </p>


-  <p style="font-size: 17px; color: black;"> Geant4 fundamental units: </p>

 -  <p style="font-size: 17px; color: black;"> Length [L]: mm (milimeter) </p>
 -  <p style="font-size: 17px; color: black;"> Time [T]: ns (nanosecond) </p>
 -  <p style="font-size: 17px; color: black;"> Energy [E]: MeV (megaelectronvolt) </p>



In [1]:
import numpy              as np
import pandas             as pd

In [2]:
import scipy              
from scipy                import stats, integrate
from scipy.signal         import convolve

In [3]:
import matplotlib.pyplot  as plt

from matplotlib.ticker    import FormatStrFormatter # to set the format of the plot's axis
from matplotlib.patches   import Rectangle # To add blanck spaces in tabular legends

In [4]:
import os
import math
import tables             as tb

<h1> _________________________________________________________________________________________________________ </h1>


<h1> __________________________ </h1>
<h2> Functions </h2>

-  <p style="font-size: 17px; color: black;"> <b> gauss(x, a,  mu, sigma): </b> Gaussian normalized to $a$. </p>


-  <p style="font-size: 17px; color: black;"> <b> gauss_sum(x, a0,  mu0, sigma0, a1, mu1, sigma1): </b> Sum of 2 gaussians. </p>


-  <p style="font-size: 17px; color: black;"> <b> dirac(x, x0): </b> Dirac delta at $x0$. </p>




In [5]:
# Fitting distributions to define

# note: pdf are normilized to 1 so we have yo add an amplitude param
def gauss(x, a,  mu, sigma):
    
    gaussian = stats.norm.pdf(np.sort(x), mu, sigma) 
    
    return (a/gaussian.sum())*gaussian

def gauss_sum(x, a0,  mu0, sigma0, a1, mu1, sigma1):
    return a0*stats.norm.pdf(x, mu0, sigma0)  + a1*stats.norm.pdf(x, mu1, sigma1)

def dirac(x, x0):
    return np.where((x - x0) == 0, 1, 0)


<h1> __________________________ </h1>
<h2> Global parameters </h2>

-  <p style="font-size: 17px; color: black;"> <b> Pandas dataset params: </b> Allows to set the max number of columns and rows that are shown in a pandas dataset. </p>

In [6]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)
pd.set_option('display.min_rows', 100)

-  <p style="font-size: 17px; color: black;"> <b> Plots params: </b> In case you want to fix the parameters for all plots. </p>

In [7]:
# plt.rcParams["figure.figsize"] = 10, 8
# plt.rcParams["font.size"] = 10

-  <p style="font-size: 17px; color: black;"> <b> Analysis parameters </b> </p>

 -  <p style="font-size: 17px; color: black;"> <b> patata: </b> patata. </p>
 


In [8]:
dt = 1e3 # [ns] = 1us


<h1> __________________________ </h1>
<h2> Data </h2>

-  <p style="font-size: 17px; color: black;"> <b> Reading the file </b> </p>

-  <p style="font-size: 17px; color: black;"> File's path </p>

In [9]:
path = '/home/investigator/mariandbt/nexus/'

filename = os.path.join(path, "FiberBarrel.next.h5")

-  <p style="font-size: 17px; color: black;"> In a .h5 file there are several objects, so we print them all to then pick the one we're interested in </p>

In [10]:
with tb.open_file(filename) as file:
    print(file)

/home/investigator/mariandbt/nexus/FiberBarrel.next.h5 (File) ''
Last modif.: '2023-07-19T08:43:35+00:00'
Object Tree: 
/ (RootGroup) ''
/MC (Group) ''
/MC/configuration (Table(33,)) ''
/MC/hits (Table(0,)) ''
/MC/particles (Table(24088,)) ''
/MC/sns_positions (Table(0,)) ''
/MC/sns_response (Table(0,)) ''



-  <p style="font-size: 17px; color: black;"> Read the file and make a copy to work with, this way it's easier to re-start the copy instead of re-reading the file. </p>

In [11]:
# %%timeit # WITH THIS YOU'RE ONLY TIMING, BUT IT DOESN'T REALLY RUN THE COMMANDS YOU TIME
data = pd.read_hdf(filename, "/MC/particles")


In [12]:
dst = data.copy()

In [13]:
dst

,event_id,particle_id,particle_name,primary,mother_id,initial_x,initial_y,initial_z,initial_t,final_x,final_y,final_z,final_t,initial_volume,final_volume,initial_momentum_x,initial_momentum_y,initial_momentum_z,final_momentum_x,final_momentum_y,final_momentum_z,kin_energy,length,creator_proc,final_proc
0,0,1,opticalphoton,1,0,-1.611572,0.797046,-3.762810,0.000000,3.604101,-2.872316,-3.156505,0.038169,WORLD,B2-28,-4.016314e-06,-5.727570e-06,2.533102e-07,6.951690e-06,-8.102241e-07,-1.324228e-07,0.000007,11.442699,none,OpWLS
1,1,1,opticalphoton,1,0,-3.019941,0.220798,6.968771,0.000000,4.230846,-1.628935,4.804766,0.027182,WORLD,B2-29,6.228468e-06,-1.588931e-06,2.771550e-06,5.891126e-06,-1.763065e-06,-3.344583e-06,0.000007,8.148956,none,OpWLS
2,1,3,opticalphoton,0,1,4.230846,-1.628935,4.804766,2.427182,4.405507,-1.802553,5.252627,2.429850,B2-29,B2,1.023503e-06,-1.219039e-06,3.095951e-06,1.287221e-06,-1.271329e-06,2.974107e-06,0.000003,0.513041,OpWLS,OpWLS
3,1,2,opticalphoton,0,1,4.230846,-1.628935,4.804766,2.427182,4.703335,-0.737712,6.074307,2.434249,B2-29,B2,3.604882e-07,2.074687e-07,3.266017e-06,1.398123e-06,1.866754e-06,2.323864e-06,0.000003,1.642610,OpWLS,OpWLS
4,1,5,opticalphoton,0,2,4.703335,-0.737712,6.074307,10.934249,4.999036,-1.024366,7.500000,10.945668,B2,DISKL-30,7.662417e-07,-2.756313e-06,6.822743e-07,2.127748e-08,1.364663e-06,2.605212e-06,0.000003,2.187354,OpWLS,Transportation
5,1,4,opticalphoton,0,2,4.703335,-0.737712,6.074307,10.934249,5.150669,-0.613346,-7.500000,11.015559,B2,WORLD,-1.269485e-06,-7.228002e-07,-2.439736e-06,-1.095285e-06,3.295512e-07,-2.603477e-06,0.000003,15.568279,OpWLS,Transportation
6,2,1,opticalphoton,1,0,0.155220,-0.092032,-6.933284,0.000000,-4.677182,1.005651,-6.371683,0.016636,WORLD,B2-14,-6.782694e-06,1.540685e-06,7.882570e-07,-5.859434e-06,3.792913e-06,5.299492e-07,0.000007,4.987226,none,OpWLS
7,2,4,opticalphoton,0,1,-4.677182,1.005651,-6.371683,2.416636,-4.742686,1.141369,-6.314115,2.417472,B2-14,B2,-2.095764e-06,-1.991966e-06,1.465371e-06,-1.296404e-06,2.747648e-06,1.130146e-06,0.000003,0.161396,OpWLS,OpWLS
8,2,5,opticalphoton,0,4,-4.742686,1.141369,-6.314115,10.917472,-4.820432,1.786532,-2.175124,10.940411,B2,B2,-6.832346e-07,-5.659431e-07,2.925939e-06,8.484445e-07,2.593125e-07,2.925939e-06,0.000003,4.325075,OpWLS,OpWLS
9,2,3,opticalphoton,0,1,-4.677182,1.005651,-6.371683,2.416636,-5.213670,-0.944846,-6.358222,2.427201,B2-14,WORLD,-2.708452e-07,-3.176188e-06,1.094830e-07,-9.573238e-07,-3.004068e-06,4.823095e-07,0.000003,2.045990,OpWLS,Transportation


In [14]:
np.shape(dst), len(dst)

((24088, 25), 24088)

In [15]:
len(dst.loc[dst['final_volume'].str.contains('DISKL')])

1280

In [16]:
print(len(dst.loc[dst['final_volume'].str.contains('DISKL')]))
print(len(dst.loc[(dst['final_volume'].str.contains('DISKL')) & (dst['initial_volume'].str.contains('B2'))]))

1280
1265


In [17]:
dst.loc[dst['final_volume'].str.contains('DISKL')]

,event_id,particle_id,particle_name,primary,mother_id,initial_x,initial_y,initial_z,initial_t,final_x,final_y,final_z,final_t,initial_volume,final_volume,initial_momentum_x,initial_momentum_y,initial_momentum_z,final_momentum_x,final_momentum_y,final_momentum_z,kin_energy,length,creator_proc,final_proc
4,1,5,opticalphoton,0,2,4.703335,-0.737712,6.074307,10.934249,4.999036,-1.024366,7.500000,10.945668,B2,DISKL-30,7.662417e-07,-2.756313e-06,6.822743e-07,2.127748e-08,1.364663e-06,2.605212e-06,2.941069e-06,2.187354,OpWLS,Transportation
33,11,5,opticalphoton,0,2,2.960686,4.268279,5.819358,10.911413,2.859379,4.453214,7.500000,11.082453,B2,DISKL-5,-3.609068e-07,7.921794e-07,-1.476822e-06,-6.824026e-07,5.404893e-07,1.476822e-06,1.714294e-06,32.551090,OpWLS,Transportation
60,21,2,opticalphoton,0,1,-2.263394,-3.992822,4.219279,2.409205,-2.101974,-4.290579,7.500000,2.450648,B2-21,DISKL-21,1.745558e-07,-2.171487e-07,-3.474585e-07,-4.025422e-07,-5.859525e-08,1.813203e-07,4.453658e-07,7.884365,OpWLS,Transportation
65,24,1,opticalphoton,1,0,-2.286224,4.201234,-3.838805,0.000000,-1.858396,4.404724,7.500000,0.063854,B2,DISKL-10,-1.062616e-06,2.099938e-06,6.592504e-06,-2.187439e-06,-8.683331e-07,6.592504e-06,7.000000e-06,12.039680,none,Transportation
103,39,5,opticalphoton,0,2,-4.353670,-1.967735,-3.172592,10.924023,-3.912699,-2.400690,7.500000,10.999249,B2,DISKL-18,1.861929e-06,-1.248650e-07,2.131828e-06,-5.549865e-09,-1.866103e-06,2.131828e-06,2.833207e-06,14.183910,OpWLS,Transportation
104,39,4,opticalphoton,0,2,-4.353670,-1.967735,-3.172592,10.924023,-4.074937,-2.657644,7.500000,10.991263,B2,DISKL-18,-9.027085e-07,9.974744e-07,2.020680e-06,1.258184e-06,4.762464e-07,2.020680e-06,2.427548e-06,12.747465,OpWLS,Transportation
120,48,1,opticalphoton,1,0,-2.019394,-4.102273,4.758967,0.000000,-2.640619,-4.530650,7.500000,0.258713,B2,DISKL-21,-5.285175e-06,2.401146e-06,-3.911704e-06,-8.604375e-07,-5.740925e-06,3.911704e-06,7.000000e-06,48.779961,none,Transportation
140,57,1,opticalphoton,1,0,3.132959,3.787304,4.154340,0.000000,3.775533,3.735938,7.500000,0.019882,B2,DISKL-4,-2.097948e-06,-2.360131e-06,6.247271e-06,-1.608600e-06,2.717353e-06,6.247271e-06,7.000000e-06,3.748776,none,Transportation
148,61,7,opticalphoton,0,6,-1.611868,4.816688,6.953571,21.812199,-0.903992,4.946897,7.500000,21.818274,B2,DISKL-9,-5.320089e-07,2.615303e-07,5.409838e-07,7.266783e-07,4.883004e-08,3.371192e-07,8.025553e-07,1.154537,OpWLS,Transportation
151,61,9,opticalphoton,0,3,-0.815884,4.598139,7.131625,7.205334,-1.353484,4.409382,7.500000,7.210069,B2-9,DISKL-9,-3.056828e-06,-1.573059e-06,1.136942e-06,2.430691e-06,1.050629e-06,2.469670e-06,3.620960e-06,0.917725,OpWLS,Transportation


In [20]:
dst.loc[(dst['final_volume'].str.contains('DISKL')) & (~dst['initial_volume'].str.contains('B2'))]

,event_id,particle_id,particle_name,primary,mother_id,initial_x,initial_y,initial_z,initial_t,final_x,final_y,final_z,final_t,initial_volume,final_volume,initial_momentum_x,initial_momentum_y,initial_momentum_z,final_momentum_x,final_momentum_y,final_momentum_z,kin_energy,length,creator_proc,final_proc
709,283,1,opticalphoton,1,0,-1.821747,2.441791,5.311253,0.0,-3.085115,3.454483,7.563200,0.009252,WORLD,DISKL-11,-3.188473e-06,0.000003,0.000006,-3.188473e-06,2.555819e-06,0.000006,0.000007,2.773609,none,Transportation
2914,1172,1,opticalphoton,1,0,-2.454705,3.718781,5.101877,0.0,1.326476,4.402650,7.507502,0.015122,WORLD,DISKL-6,5.838457e-06,0.000001,0.000004,5.838457e-06,1.055950e-06,0.000004,0.000007,4.533436,none,Transportation
2955,1190,1,opticalphoton,1,0,3.041414,-1.258667,6.306108,0.0,4.625738,0.586893,7.558191,0.009125,WORLD,DISKL-1,4.053951e-06,0.000005,0.000003,4.053951e-06,4.722398e-06,0.000003,0.000007,2.735669,none,Transportation
6418,2600,1,opticalphoton,1,0,1.890762,-3.366471,6.223100,0.0,2.944877,-3.630463,7.531939,0.005674,WORLD,DISKL-27,4.337540e-06,-0.000001,0.000005,4.337540e-06,-1.086291e-06,0.000005,0.000007,1.701150,none,Transportation
6619,2679,1,opticalphoton,1,0,-2.141997,0.526194,4.332759,0.0,-0.466201,4.541200,7.677877,0.018973,WORLD,DISKL-8,1.732842e-06,0.000005,0.000004,5.344665e-06,1.711435e-07,0.000005,0.000007,5.687840,none,Transportation
6994,2824,1,opticalphoton,1,0,-0.791162,-2.387090,-4.877705,0.0,0.364368,4.632937,7.605960,0.047929,WORLD,DISKL-7,5.629423e-07,0.000003,0.000006,5.629423e-07,3.419960e-06,0.000006,0.000007,14.368644,none,Transportation
11693,4782,1,opticalphoton,1,0,2.436270,-3.002953,-3.657070,0.0,-2.287446,3.996383,7.648066,0.069327,WORLD,DISKL-10,-1.590959e-06,0.000002,-0.000006,-1.590959e-06,2.357393e-06,0.000006,0.000007,20.783699,none,Transportation
13487,5547,1,opticalphoton,1,0,-2.246804,-1.984042,5.320872,0.0,0.863421,-4.453768,7.507417,0.015123,WORLD,DISKL-24,4.802213e-06,-0.000004,0.000003,4.802213e-06,-3.813278e-06,0.000003,0.000007,4.533655,none,Transportation
14655,6059,1,opticalphoton,1,0,-0.553881,-2.069673,4.713323,0.0,4.521163,1.335518,7.578064,0.022514,WORLD,DISKL-1,5.263260e-06,0.000004,0.000003,5.263260e-06,3.531479e-06,0.000003,0.000007,6.749677,none,Transportation
15036,6215,1,opticalphoton,1,0,-0.648178,4.380606,-0.008738,0.0,3.643995,-3.052717,7.644886,0.038361,WORLD,DISKL-28,2.612578e-06,-0.000005,0.000005,2.612578e-06,-4.524546e-06,0.000005,0.000007,11.500217,none,Transportation


In [19]:
dst['length'].mean()

10.07233715057373